In [ ]:
import os
import sys
import numpy as np
import torch

from google.colab import drive

try:
    drive.mount('/content/drive')
except:
    pass

project_root = "/content/drive/MyDrive/Colab Notebooks/ecg-augmentation-repro-minkuek2"
data_dir = os.path.join(project_root, "data")
src_dir = os.path.join(project_root, "src")
results_dir = os.path.join(project_root, "results")

os.makedirs(results_dir, exist_ok=True)

if src_dir not in sys.path:
    sys.path.append(src_dir)

print("Project root:", project_root)
print("Data dir    :", data_dir)
print("SRC dir     :", src_dir)
print("Train CSV   :", os.path.exists(os.path.join(data_dir, "mitbih_train.csv")))
print("Test  CSV   :", os.path.exists(os.path.join(data_dir, "mitbih_test.csv")))

In [ ]:
from dataset import create_dataloaders
from model import ECGClassifier
from train import train_one_epoch, evaluate

# GPU / CPU configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Create DataLoaders
train_loader, val_loader, test_loader = create_dataloaders(
    data_dir=data_dir,
    batch_size=128,
    val_ratio=0.2,
    seed=42,
)

len(train_loader.dataset), len(val_loader.dataset), len(test_loader.dataset)

In [ ]:
num_classes = 5
input_length = 187

model = ECGClassifier(input_length=input_length, num_classes=num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

print(model)

In [ ]:
num_classes = 5
input_length = 187

model = ECGClassifier(input_length=input_length, num_classes=num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

print(model)

In [ ]:
import json

num_epochs = 20

history = {
    "train_loss": [],
    "val_loss": [],
    "train_acc": [],
    "val_acc": [],
}

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer, device
    )
    val_loss, val_acc = evaluate(
        model, val_loader, criterion, device
    )

    history["train_loss"].append(train_loss)
    history["val_loss"].append(val_loss)
    history["train_acc"].append(train_acc)
    history["val_acc"].append(val_acc)

    print(
        f"Epoch {epoch:02d}/{num_epochs} "
        f"| train_loss={train_loss:.4f}, val_loss={val_loss:.4f} "
        f"| train_acc={train_acc:.4f}, val_acc={val_acc:.4f}"
    )

model_path = os.path.join(results_dir, "baseline_cnn.pt")
torch.save(model.state_dict(), model_path)
print("Saved baseline model to:", model_path)

history_path = os.path.join(results_dir, "baseline_metrics.json")
with open(history_path, "w") as f:
    json.dump(history, f, indent=2)
print("Saved baseline metrics to:", history_path)

In [ ]:
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"Test loss: {test_loss:.4f}, Test acc: {test_acc:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import os

# Ensure directory exists
fig_dir = os.path.join(project_root, "results", "figures")
os.makedirs(fig_dir, exist_ok=True)

# Suppose these lists exist from your training loop
# history["train_loss"] and history["val_loss"]
# If your variable names differ, adjust accordingly.

plt.figure(figsize=(8,4))
plt.plot(history["train_loss"], label="Train Loss")
plt.plot(history["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Baseline CNN Training/Validation Loss Curve")
plt.legend()
plt.grid(True)

save_path = os.path.join(fig_dir, "loss_curves.png")
plt.savefig(save_path, dpi=200, bbox_inches="tight")
print("Saved loss curve to:", save_path)
plt.show()

In [ ]:
import os
import matplotlib.pyplot as plt

project_root = "/content/drive/MyDrive/Colab Notebooks/ecg-augmentation-repro-minkuek2"
fig_dir = os.path.join(project_root, "results", "figures")
os.makedirs(fig_dir, exist_ok=True)

sigmas = [0.01, 0.03, 0.05, 0.10]
val_accs = [0.973, 0.977, 0.974, 0.958]

plt.figure(figsize=(6,4))
plt.plot(sigmas, val_accs, marker='o', linewidth=2)
plt.title("Jitter Strength Ablation (σ vs Validation Accuracy)")
plt.xlabel("Jitter Sigma (σ)")
plt.ylabel("Validation Accuracy")
plt.grid(True)

save_path = os.path.join(fig_dir, "jitter_ablation.png")
plt.savefig(save_path, dpi=150)
plt.show()

print("Saved figure to:", save_path)